In [2]:
import numpy as np
import xarray as xr
import os
from cdo import *
from cdo import Cdo
cdo = Cdo()

import geopandas as geo
import salem


In [3]:
#导入保护区的shp
reserve_shp = geo.read_file('/mnt/e/BaiduSyncdisk/lunwen/毕设的糟心玩意/华北国家级保护区shp/华北保护区.shp')
#导入需要裁剪的cn05/NDVI数据
#此处数据都是已经salem裁剪成华北地区，box裁剪成小内存的了

#cn05包括降水和气温，降水由于后面做过改正，所以是另外导入的new_pre
#有一部分降水指数已经输出成文件了所以直接裁剪

cn05_pre = '/mnt/g/cn05/new_pre/pre/'
cn05_r1mm = '/mnt/g/cn05/new_pre/R1mm/'
cn05_r10mm = '/mnt/g/cn05/new_pre/R10mm/'
cn05_r20mm = '/mnt/g/cn05/new_pre/R20mm/'
cn05_rx1day = '/mnt/g/cn05/new_pre/Rx1day/'

cn05_tmax = '/mnt/g/cn05/cn05_huabei/tmax/'
cn05_tmin = '/mnt/g/cn05/cn05_huabei/tmin/'

pre_data = [cn05_pre,cn05_r1mm,cn05_r10mm,cn05_r20mm,cn05_rx1day]
tm_data = [cn05_tmax,cn05_tmin]


i = 0
for data in pre_data:
    filename = os.listdir(data)
    for file in filename:
        inputfile = os.path.join(data,file)
        input = xr.open_dataset(inputfile)
        if i == 0:
            input = input.pre
        elif i == 4:
            input = input.highest_one_day_precipitation_amount_per_time_period
        else:
            input = input.precipitation_days_index_per_time_period
        output = input.salem.roi(shape = reserve_shp)
        outputfile = os.path.join('/mnt/g/reserve/{}',file).format(pre_data[i][20:])
        output.to_netcdf(path = outputfile)
    i = i + 1

i = 0
for data in tm_data:
    filename = os.listdir(data)
    for file in filename:
        inputfile = os.path.join(data,file)
        input = xr.open_dataset(inputfile)
        if i == 0:
            input = input.tmax
        else:
            input = input.tmin
        output = input.salem.roi(shape = reserve_shp)
        outputfile = os.path.join('/mnt/g/reserve/{}',file).format(tm_data[i][23:])
        output.to_netcdf(path = outputfile)
    i = i + 1

#NDVI包括插值前和插值后的，插值前用来做趋势分析，插值后做相关分析
ndvi = '/mnt/g/month/'
ndvi_remapbic = '/mnt/g/month_remapbic'

ndvi_data = [ndvi,ndvi_remapbic]

i = 0
for data in ndvi_data:
    filename = os.listdir(data)
    for file in filename:
        inputfile = os.path.join(data,file)
        input = xr.open_dataset(inputfile)
        input = input.ndvi
        output = input.salem.roi(shape = reserve_shp)
        outputfile = os.path.join('/mnt/g/reserve/ndvi{}',file).format(ndvi_data[i][12:])
        output.to_netcdf(path = outputfile)
        print(outputfile)
    i = i + 1

"\ni = 0\nfor data in pre_data:\n    filename = os.listdir(data)\n    for file in filename:\n        inputfile = os.path.join(data,file)\n        input = xr.open_dataset(inputfile)\n        if i == 0:\n            input = input.pre\n        elif i == 4:\n            input = input.highest_one_day_precipitation_amount_per_time_period\n        else:\n            input = input.precipitation_days_index_per_time_period\n        output = input.salem.roi(shape = reserve_shp)\n        outputfile = os.path.join('/mnt/g/reserve/{}',file).format(pre_data[i][20:])\n        output.to_netcdf(path = outputfile)\n    i = i + 1\n"